Section 1: Import Libraries

In [10]:
import requests                                                             # This lib helps us to fetch data from API
import pandas as pd                                                          # For handling and analysing data
import numpy as np                                                           # For numerical operations
from sklearn.model_selection import train_test_split                         # This is used to split data for training and testing
from sklearn.preprocessing import LabelEncoder                               # It is use for converting categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor   # Models for classification and regression task
from sklearn.metrics import mean_squared_error                               # To measure the accuracy of our prediction
from datetime import datetime,timedelta                                      # To handle date and time
import pytz



In [ ]:
API_KEY='Your_Api' #replace with your actual API key
BASE_URL='https://api.openweathermap.org/data/2.5/' #Base url for making API req

1. Fetch Current Weather Data

In [2]:
def get_current_weather(city):
    url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"  #Construct the API req url
    response=requests.get(url)  #send the get request to API
    data=response.json()
    return{
        'city':data['name'],
        'current_temp':round(data['main']['temp']),
        'feels_like':round(data['main']['feels_like']),
        'temp_min':round(data['main']['temp_min']),
        'temp_max':round(data['main']['temp_max']),
        'humidity':round(data['main']['humidity']),
        'description':(data['weather'][0]['description']),
        'country':data['sys']['country'],
        'wind_gust_dir':data['wind']['deg'],
        'pressure':data['main']['pressure'],
        'Wind_Gust_Speed':data['wind']['speed']

    }


2.Read Historical Data

In [3]:
def read_historical_data(filename):
    df=pd.read_csv(filename)             # Load csv file  into dataframe
    df=df.dropna()
    df=df.drop_duplicates()
    return df


3. Prepare data for training

In [4]:
def prepare_data(data):  # Accept DataFrame instead of filename
    le = LabelEncoder()  # Create a LabelEncoder instance
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])  
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    # Define the feature variable and target variable
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
    y = data['RainTomorrow']

    return X, y, le

4. Train Rain Prediciton Model

In [5]:
def train_rain_model(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    model=RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(X_train,y_train)          #Train the model

    y_pred=model.predict(X_test)        #To make prediction on test set

    print("Mean squared error for rain model")

    print(mean_squared_error(y_test,y_pred))

    return model


5. Prepare regression data


In [6]:
def prepare_regression_data(data,feature):
    X,y=[],[]  #intialise  list for feature and target variable

    for i in range(len(data)-1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])
    
    X=np.array(X).reshape(-1,1)
    y=np.array(y)

    return X,y

6. Train Regression Model

In [7]:
def train_regression_model(X,y):
    model=RandomForestRegressor(n_estimators=100,random_state=42)
    model.fit(X,y)
    return model

7. Predict Future

In [8]:
def predict_future(model,current_value):
    predictions=[current_value]

    for i in range(5):
        next_value=model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    return predictions[1:]


8.Weather Analysis Function

In [11]:
def weather_view():
    city = input("Enter city Name:")
    current_weather = get_current_weather(city)

    # Load historical data
    historical_data = pd.read_csv('weather.csv')  #  Load dataset
    X, y, le = prepare_data(historical_data)  #  Pass the DataFrame directly

    # Train the rain prediction model
    rain_model = train_rain_model(X, y)

    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360

    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25), ("ENE", 56.25, 78.75),
        ("E", 78.75, 101.25), ("ESE", 101.25, 123.75), ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75),
        ("S", 168.75, 191.25), ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25), ("NNW", 326.25, 348.75)
    ]
    
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['Wind_Gust_Speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }

    current_df = pd.DataFrame([current_data])

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')  
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

    # Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    # Prepare time for future prediction
    timezone = pytz.timezone('Asia/Kolkata')  
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

    future_times = [(next_hour + timedelta(hours=i)).strftime('%H:00') for i in range(5)]

    # Display results
    print(f'City: {city}, {current_weather["country"]}')
    print(f'Current Temperature: {current_weather["current_temp"]}°C')
    print(f'Feels like: {current_weather["feels_like"]}°C')
    print(f'Minimum Temperature: {current_weather["temp_min"]}°C')
    print(f'Maximum Temperature: {current_weather["temp_max"]}°C')
    print(f'Humidity: {current_weather["humidity"]}%')
    print(f'Weather Prediction: {current_weather["description"]}')
    print(f'Rain Prediction: {"Yes" if rain_prediction else "No"}')

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

weather_view()

Mean squared error for rain model
0.16216216216216217
City: Pune, IN
Current Temperature: 32°C
Feels like: 30°C
Minimum Temperature: 32°C
Maximum Temperature: 32°C
Humidity: 15%
Weather Prediction: clear sky
Rain Prediction: No

Future Temperature Predictions:
13:00: 32.5°C
14:00: 32.0°C
15:00: 32.7°C
16:00: 31.1°C
17:00: 31.0°C

Future Humidity Predictions:
13:00: 21.1%
14:00: 27.8%
15:00: 27.8%
16:00: 27.8%
17:00: 27.8%
